# Weather Data

## Loading Env Variables

In [64]:
from configparser import ConfigParser
job_name = 'influx viz'
path = '/home/armando/Github Repos/weatherAPI_data_eng/src/'

config_object = ConfigParser(interpolation = None)
config_object.read(path + 'config.ini')

influx_db = config_object['DATABASE']    
my_token = influx_db['token']
my_org = influx_db['organization']
my_agg = influx_db['agg_bucket']

## Loading the aggregations

In [65]:
from influxdb_client import InfluxDBClient

with InfluxDBClient(url='https://us-east-1-1.aws.cloud2.influxdata.com', token=my_token, org=my_org) as client:
        # start 0, brings all available data
        # pivot, structures the influx tables into a single df
        # measurement filter, so that when I add new measures this job isn't affected    
        q = f""" from(bucket:"{my_agg}") 
                |> range(start:0)
                |> filter(fn: (r) => r["_measurement"] == "weather_aggregations")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") """
        df_query = client.query_api().query_data_frame(q)
        df_query.drop(["table", "result", "_start", "_stop", "_measurement"], inplace=True, axis=1)
        client.close()

In [66]:
df_query = df_query.sort_values(by=['_time'], ascending=False)

print(df_query[(df_query["city"] == "Monterrey") & (df_query["window_time"] == "12H") & (df_query["agg"] == "mean")].head())

                        _time   agg       city country weather_class  \
317 2022-04-15 12:00:00+00:00  mean  Monterrey      MX        Clouds   
316 2022-04-15 00:00:00+00:00  mean  Monterrey      MX        Clouds   

    window_time  cloudiness   humidity     pressure  sensation  temperature  \
317         12H   99.947368  74.105263  1011.842105  23.285789    22.694211   
316         12H  100.000000  69.727273  1010.454545  21.303636    21.299091   

     visibility  
317     10000.0  
316     10000.0  


In [67]:

timezones = {"Monterrey":"America/Mexico_City", 
             "Boston": "America/NewYork", 
             "Saltillo":"America/Monterrey", 
             "Guadalajara":"America/Monterrey", 
             "Mexico City":"America/Monterrey"}

city = "Monterrey"
df_query.set_index("_time", inplace=True)
df_query.sort_index(inplace = True)
df_query.index = df_query.index.tz_convert(timezones[city])

print(df_query[(df_query["city"] == "Monterrey") & (df_query["window_time"] == "12H") & (df_query["agg"] == "mean")].head())

                            agg       city country weather_class window_time  \
_time                                                                          
2022-04-14 19:00:00-05:00  mean  Monterrey      MX        Clouds         12H   
2022-04-15 07:00:00-05:00  mean  Monterrey      MX        Clouds         12H   

                           cloudiness   humidity     pressure  sensation  \
_time                                                                      
2022-04-14 19:00:00-05:00  100.000000  69.727273  1010.454545  21.303636   
2022-04-15 07:00:00-05:00   99.947368  74.105263  1011.842105  23.285789   

                           temperature  visibility  
_time                                               
2022-04-14 19:00:00-05:00    21.299091     10000.0  
2022-04-15 07:00:00-05:00    22.694211     10000.0  


### Seaborn / Streamlit